In [10]:
import pathlib
import scanpy as sc
import scvi

/home/victor/anaconda3/envs/rna/lib/python3.10/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/victor/anaconda3/envs/rna/lib/python3.10/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/home/victor/anaconda3/envs/rna/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
SAMPLE_RAW_DATA_PATH = DATA_DIR / 'raw_data' / 'GSM5226574_C51ctr_raw_counts.csv.gz'

In [4]:
sample_data = sc.read_csv(SAMPLE_RAW_DATA_PATH).T
sample_data

AnnData object with n_obs × n_vars = 6099 × 34546

## Doublet removal

In [5]:
# Filter genes expressed in less than 10 cells
sc.pp.filter_genes(sample_data, min_cells=10)

In [7]:
# Keep only top 2000 highly variable genes
sc.pp.highly_variable_genes(sample_data, n_top_genes=2000, subset=True, flavor='seurat_v3')

In [12]:
scvi.model.SCVI.setup_anndata(sample_data)
model = scvi.model.SCVI(sample_data)
model.train()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Epoch 22/400:   5%|▌         | 21/400 [00:21<06:30,  1.03s/it, v_num=1, train_loss_step=386, train_loss_epoch=341]

/home/victor/anaconda3/envs/rna/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
